In [147]:
import pandas as pd

class AcsData:
    def __init__(self,filepath="data/acs.csv"):
        self.filepath = filepath
        self.df = self.load_data()
        self.rename_columns()
        self.summary = self.create_summary()

    def create_summary(self):
        races = ['white', 'black', 'other', 'asian','hispanic']
        
        summary = self.df.groupby('county_fips').sum()
        for race in races:
            col_name = race + '_percentage'
            summary[col_name] = summary[race] / summary['total_population']

        return summary

    def rename_columns(self):
        fips = self.df['GEO.id2'].astype('str').str[:5]

        hispanic_values = ['HD01_VD12','HD01_VD13','HD01_VD14','HD01_VD15','HD01_VD16','HD01_VD17','HD01_VD18',
                           'HD01_VD19','HD01_VD20','HD01_VD21']

        race_cols = ['HD01_VD01','HD01_VD03','HD01_VD04','HD01_VD06']
        native_cols = ['HD01_VD05','HD01_VD07','HD01_VD08']

        combined_cols = hispanic_values + race_cols + native_cols
        df = self.df[combined_cols]
        df[combined_cols] = df[combined_cols].astype(int)
        df['county_fips'] = fips

        df['hispanic'] = df[hispanic_values].sum(axis=1)
        df['other'] = df[native_cols].sum(axis=1)

        df.drop(hispanic_values,axis=1,inplace=True)
        df.drop(native_cols, axis=1, inplace=True)
        col_names = {
            'HD01_VD01': 'total_population',
            'HD01_VD03': 'white',
            'HD01_VD04': 'black',
            'HD01_VD06': 'asian',
            'fips': 'fips',
            'hispanic':'hispanic',
            'other':'other'
        }

        df.rename(columns=col_names, inplace=True)
        self.df = df

    def load_data(self):
        community_data = pd.read_csv(self.filepath, encoding="ISO-8859-1")
        data = community_data[1:]
        return data

In [184]:
import pandas as pd

class Stop:
    def __init__(self,filepath):
        self.df = self.load_dataframe(filepath)
        self.summary = self.create_summary()

    def load_dataframe(self, filepath):
        df = pd.read_csv(filepath)
        df = df[df['county_fips'].notna()]
        df = df[df['driver_race'].notna()]
        df['driver_race'] = df['driver_race'].str.lower()
        df['county_fips'] = df['county_fips'].astype(int).astype(str)
        df['state_officer_id'] = df['state'].str.lower() + df['officer_id'].astype(int).astype(str)

        cols_to_drop = ['location_raw', 'county_name', 'driver_race_raw','officer_id']
        df = df.drop(cols_to_drop, axis=1)

        return df

    def create_summary(self):
        summary = self.add_stop_percentage_to_summary_table()
        pivot = self.create_single_columns_from_summary_table(summary)

        return pivot

    def create_single_columns_from_summary_table(self, summary):
        summary = summary.reset_index()
        melt = summary.melt(id_vars=['county_fips', 'driver_race'], value_vars=['stops', 'stop_percentage'])
        pivot = melt.pivot_table(index=['county_fips'], columns=['driver_race', 'variable'], values='value')
        pivot.columns = ['_'.join(col).strip() for col in pivot.columns.values]
        pivot.columns = pivot.columns.get_level_values(0)
        return pivot

    def add_stop_percentage_to_summary_table(self):
        summary = self.df.groupby(['county_fips', 'driver_race']).agg('count')
        summary = summary[['id']]
        individual_label = 'stops'
        summary[individual_label] = summary['id']
        summary = summary[[individual_label]]
        stop_percentage_label = 'stop_percentage'
        summary[stop_percentage_label] = summary[individual_label] / summary[individual_label].groupby(level=0).sum()
        return summary

In [185]:
filepath = 'data/stop_data/WY-clean.csv'
stops = Stop(filepath)

/Users/shawnveltman/anaconda3/envs/geo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2961: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [186]:
stops.summary

,asian_stop_percentage,asian_stops,black_stop_percentage,black_stops,hispanic_stop_percentage,hispanic_stops,other_stop_percentage,other_stops,white_stop_percentage,white_stops
county_fips,,,,,,,,,,
56001,0.017473,312.0,0.027946,499.0,0.065972,1178.0,0.002352,42.0,0.886257,15825.0
56003,0.014324,37.0,0.005033,13.0,0.036005,93.0,0.007743,20.0,0.936895,2420.0
56005,0.010294,57.0,0.011920,66.0,0.050208,278.0,0.003612,20.0,0.923966,5116.0
56007,0.023850,403.0,0.032550,550.0,0.069006,1166.0,0.003728,63.0,0.870865,14715.0
56009,0.015045,90.0,0.016382,98.0,0.063022,377.0,0.006520,39.0,0.899030,5378.0
56011,0.018311,72.0,0.021363,84.0,0.032808,129.0,0.004832,19.0,0.922686,3628.0
56013,0.016646,161.0,0.008995,87.0,0.029363,284.0,0.056658,548.0,0.888337,8592.0
56015,0.008211,37.0,0.019308,87.0,0.058589,264.0,0.005326,24.0,0.908566,4094.0
56017,0.015535,35.0,0.008433,19.0,0.019973,45.0,0.023524,53.0,0.932534,2101.0


,asian_stop_percentage,asian_stops,black_stop_percentage,black_stops,hispanic_stop_percentage,hispanic_stops,other_stop_percentage,other_stops,white_stop_percentage,white_stops
county_fips,,,,,,,,,,
56001,0.017473,312.0,0.027946,499.0,0.065972,1178.0,0.002352,42.0,0.886257,15825.0
56003,0.014324,37.0,0.005033,13.0,0.036005,93.0,0.007743,20.0,0.936895,2420.0
56005,0.010294,57.0,0.011920,66.0,0.050208,278.0,0.003612,20.0,0.923966,5116.0
56007,0.023850,403.0,0.032550,550.0,0.069006,1166.0,0.003728,63.0,0.870865,14715.0
56009,0.015045,90.0,0.016382,98.0,0.063022,377.0,0.006520,39.0,0.899030,5378.0
56011,0.018311,72.0,0.021363,84.0,0.032808,129.0,0.004832,19.0,0.922686,3628.0
56013,0.016646,161.0,0.008995,87.0,0.029363,284.0,0.056658,548.0,0.888337,8592.0
56015,0.008211,37.0,0.019308,87.0,0.058589,264.0,0.005326,24.0,0.908566,4094.0
56017,0.015535,35.0,0.008433,19.0,0.019973,45.0,0.023524,53.0,0.932534,2101.0


,asian_stop_percentage,asian_stops,black_stop_percentage,black_stops,hispanic_stop_percentage,hispanic_stops,other_stop_percentage,other_stops,white_stop_percentage,white_stops
county_fips,,,,,,,,,,
56001,0.017473,312.0,0.027946,499.0,0.065972,1178.0,0.002352,42.0,0.886257,15825.0
56003,0.014324,37.0,0.005033,13.0,0.036005,93.0,0.007743,20.0,0.936895,2420.0
56005,0.010294,57.0,0.011920,66.0,0.050208,278.0,0.003612,20.0,0.923966,5116.0
56007,0.023850,403.0,0.032550,550.0,0.069006,1166.0,0.003728,63.0,0.870865,14715.0
56009,0.015045,90.0,0.016382,98.0,0.063022,377.0,0.006520,39.0,0.899030,5378.0
56011,0.018311,72.0,0.021363,84.0,0.032808,129.0,0.004832,19.0,0.922686,3628.0
56013,0.016646,161.0,0.008995,87.0,0.029363,284.0,0.056658,548.0,0.888337,8592.0
56015,0.008211,37.0,0.019308,87.0,0.058589,264.0,0.005326,24.0,0.908566,4094.0
56017,0.015535,35.0,0.008433,19.0,0.019973,45.0,0.023524,53.0,0.932534,2101.0


In [169]:
acs = AcsData()
acs.summary

/Users/shawnveltman/anaconda3/envs/geo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2961: DtypeWarning: Columns (1,3,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/shawnveltman/anaconda3/envs/geo/lib/python3.6/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/Users/shawnveltman/anaconda3/envs/geo/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,total_population,white,black,asian,hispanic,other,white_percentage,black_percentage,other_percentage,asian_percentage,hispanic_percentage
county_fips,,,,,,,,,,,
10001,171474,108515,40690,3401,23779,1461,0.632836,0.237295,0.008520,0.019834,0.138674
10003,551997,326993,130912,28383,106542,2435,0.592382,0.237161,0.004411,0.051419,0.193012
10005,211224,158057,25948,2379,39438,1463,0.748291,0.122846,0.006926,0.011263,0.186712
10010,55049,41663,10113,444,2851,355,0.756835,0.183709,0.006449,0.008066,0.051790
10030,199510,165950,18406,1325,17856,1533,0.831788,0.092256,0.007684,0.006641,0.089499
10039,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
10059,26614,12212,12745,118,2300,244,0.458856,0.478883,0.009168,0.004434,0.086421
10070,22572,16876,4788,14,1004,81,0.747652,0.212121,0.003589,0.000620,0.044480
10090,57704,50582,899,92,10210,301,0.876577,0.015580,0.005216,0.001594,0.176937


In [181]:
merge = pd.merge(pivot, acs.summary,on='county_fips')
merge

,county_fips,asian_stop_percentage,asian_stops,black_stop_percentage,black_stops,hispanic_stop_percentage,hispanic_stops,other_stop_percentage,other_stops,white_stop_percentage,...,white,black,asian,hispanic,other,white_percentage,black_percentage,other_percentage,asian_percentage,hispanic_percentage
0,56001,0.017473,312.0,0.027946,499.0,0.065972,1178.0,0.002352,42.0,0.886257,...,31506,509,1139,7169,316,0.832699,0.013453,0.008352,0.030104,0.189476
1,56003,0.014324,37.0,0.005033,13.0,0.036005,93.0,0.007743,20.0,0.936895,...,10492,28,64,2141,143,0.879390,0.002347,0.011986,0.005364,0.179448
2,56005,0.010294,57.0,0.011920,66.0,0.050208,278.0,0.003612,20.0,0.923966,...,42547,360,305,8337,451,0.877746,0.007427,0.009304,0.006292,0.171993
3,56007,0.023850,403.0,0.032550,550.0,0.069006,1166.0,0.003728,63.0,0.870865,...,12231,195,46,5851,118,0.779243,0.012424,0.007518,0.002931,0.372770
4,56009,0.015045,90.0,0.016382,98.0,0.063022,377.0,0.006520,39.0,0.899030,...,12637,1,123,2230,54,0.888490,0.000070,0.003797,0.008648,0.156788
5,56011,0.018311,72.0,0.021363,84.0,0.032808,129.0,0.004832,19.0,0.922686,...,7047,12,10,350,7,0.967463,0.001647,0.000961,0.001373,0.048051
6,56013,0.016646,161.0,0.008995,87.0,0.029363,284.0,0.056658,548.0,0.888337,...,28559,210,168,5840,7816,0.701989,0.005162,0.192120,0.004129,0.143549
7,56015,0.008211,37.0,0.019308,87.0,0.058589,264.0,0.005326,24.0,0.908566,...,11705,20,32,2943,62,0.864093,0.001476,0.004577,0.002362,0.217260
8,56017,0.015535,35.0,0.008433,19.0,0.019973,45.0,0.023524,53.0,0.932534,...,4505,7,0,295,66,0.942271,0.001464,0.013805,0.000000,0.061703
9,56019,0.014243,53.0,0.020962,78.0,0.020425,76.0,0.006987,26.0,0.937382,...,7849,38,0,366,338,0.915656,0.004433,0.039431,0.000000,0.042697


In [ ]:
acs.summary.dtypes()

In [180]:
pivot.columns = [' '.join(col).strip() for col in df.columns.values]
pivot.columns = pivot.columns.get_level_values(0)
pivot

,asian_stop_percentage,asian_stops,black_stop_percentage,black_stops,hispanic_stop_percentage,hispanic_stops,other_stop_percentage,other_stops,white_stop_percentage,white_stops
county_fips,,,,,,,,,,
56001,0.017473,312.0,0.027946,499.0,0.065972,1178.0,0.002352,42.0,0.886257,15825.0
56003,0.014324,37.0,0.005033,13.0,0.036005,93.0,0.007743,20.0,0.936895,2420.0
56005,0.010294,57.0,0.011920,66.0,0.050208,278.0,0.003612,20.0,0.923966,5116.0
56007,0.023850,403.0,0.032550,550.0,0.069006,1166.0,0.003728,63.0,0.870865,14715.0
56009,0.015045,90.0,0.016382,98.0,0.063022,377.0,0.006520,39.0,0.899030,5378.0
56011,0.018311,72.0,0.021363,84.0,0.032808,129.0,0.004832,19.0,0.922686,3628.0
56013,0.016646,161.0,0.008995,87.0,0.029363,284.0,0.056658,548.0,0.888337,8592.0
56015,0.008211,37.0,0.019308,87.0,0.058589,264.0,0.005326,24.0,0.908566,4094.0
56017,0.015535,35.0,0.008433,19.0,0.019973,45.0,0.023524,53.0,0.932534,2101.0
